In [1]:
import pandas as pd
import numpy as np
import time

# 读取女性评分
with open("Scores_for_Women.txt", "r") as f:
    w_to_m_data = f.readlines()[1:]

# 转化为矩阵 w_to_m_score_matrix[i][j] 为 女性i对男性j的评分
w_to_m_Scores = []    
for i in range(len(w_to_m_data)):
    w_to_m_Scores.append([float(ii) for ii in w_to_m_data[i].strip('\n').split(' ')[:-1]])
    
w_to_m_score_matrix = np.array(w_to_m_Scores)

# 读取男性评分
with open("Scores_for_Men.txt", "r") as f:
    m_to_w_data = f.readlines()[1:]

# 转化为矩阵 m_to_w_score_matrix[i][j] 为 男性i对女性j的评分    
m_to_w_Scores = []    
for i in range(len(m_to_w_data)):
    m_to_w_Scores.append([float(ii) for ii in m_to_w_data[i].strip('\n').split(' ')[:-1]])
    
m_to_w_score_matrix = np.array(m_to_w_Scores)


# 读取拒绝列表
with open("Reject_List.txt", "r") as f:
    reject_data = f.readlines()[1:]

# 转化为列表 
reject_list = []    
for i in range(len(reject_data)):
    reject_list.append([int(ii) for ii in reject_data[i].strip('\n').split(' ')])
    
# print('reject_list', reject_list)

W_dict = {} # 女性可接受词典
M_dict = {} # 男性可接受词典

for i in range(68):
    W_dict[i] = [i for i in range(68)]
    M_dict[i] = [i for i in range(68)]

for j in reject_list: # 按拒绝列表移除不接受的用户
    W_dict[j[0]].remove(j[1])
    M_dict[j[1]].remove(j[0])

## Man-optimal GS

In [5]:
#Inputs: 
#men_to_women_scores[i][j] = k means man i rates woman j with score k
#women_to_men_score[i][j] = k  means woman i rates man j with score k

#TODO: If man i cannot be matched with woman j, then score between them be None

#Outputs:
#A dictionary matching
#matching[('m',i)] = ('w', j) means man i is matched to woman j
#matching[('m',i)] = ('m', j) means man i is matched to himself, i.e., man i is unmatched. Same for woman.

men_to_women_scores = m_to_w_Scores
women_to_men_scores = w_to_m_Scores

numOfMen = len(men_to_women_scores)
numOfWomen = len(women_to_men_scores)

In [33]:
#For every man/women i, it holds a list preference_rank_man/women
#preference_rank_man/woman[0] always holds his/her favorable acceptable woman/man
#If preference_rank_man/woman[0] = -1, then he/she prefers to be alone

#preference_rank_men is a dictionary that maps man i to his preference_list_man
preference_rank_men = {}
for i in range(numOfMen):
    scores = men_to_women_scores[i] #Get man i 's rating for woman
    preference_rank_man, scores_rank= zip(*sorted(list(zip(range(len(scores)),scores)),key = lambda x:x[1],reverse = True))
    preference_rank_man = list(preference_rank_man)
    #Remove unacceptable
    for j in preference_rank_man:
        if j not in M_dict[i]:
            preference_rank_man.remove(j)
    #Add -1 at the end
    preference_rank_man.append(-1)
    preference_rank_men[i] = preference_rank_man

In [41]:
#Some examination of the first man
print(preference_rank_men[0])
len(preference_rank_men[0])

[60, 7, 26, 40, 51, 56, 4, 14, 27, 16, 11, 15, 31, 32, 34, 37, 39, 41, 46, 1, 3, 8, 10, 17, 20, 21, 25, 28, 29, 30, 48, 53, 61, 62, 64, 22, 12, 0, 2, 13, 33, 35, 49, 52, 54, 67, 38, 55, 9, 65, 18, 43, 57, 59, 47, 42, 23, 6, 58, 44, 50, 66, 24, 19, 45, 63, -1]


67

In [52]:
#Initiate the output dictionary matching
matching = {}

#Initially, let every woman proposed to herself
for i in range(numOfWomen):
    matching[('w', i)] = ('w',i)

#We will manage a list of unmatched men
unmatchedMen = list(range(numOfMen))

while len(unmatchedMen) != 0:
    #We first get the Index of a unmatched man
    #Note that which man we pick here is not important as we will always arrive at a stable matching
    #Furthermore, if the preferences are strict, the man-optimal stable matching is unique(Lattice Theorem)
    manIndex = unmatchedMen.pop()
    
    #Get the man's preference rank
    preference_rank_man = preference_rank_men[manIndex]
    
    #Remove and store the index of his next favorite person
    proposeNext = preference_rank_man.pop(0)
    
    #If the man prefers to be alone, let him be :)
    if proposeNext == -1:
        matching[('m', manIndex)] = ('m', manIndex)
        continue
    
    #If the man still wants to propose to some woman
    else:
        
        #If the woman is matched to herself, match the man to his next favorite woman
        #Note that we don't need to consider unacceptability here as we have managed it in man's preference_rank_man
        if matching[('w', proposeNext)] == ('w', proposeNext):
            matching[('m', manIndex)] = ('w', proposeNext)
            matching[('w', proposeNext)] = ('m', manIndex)
            continue
                     
        #Otherwise if the woman is matched with some rival man
        else:
            #First get the rival's Index
            rivalIndex = matching[('w', proposeNext)][1]
            
            #If the woman prefers the rival :<
            if women_to_men_scores[proposeNext][rivalIndex] >= women_to_men_scores[proposeNext][manIndex]:
                #Then the man will still be single, i,e., add him back to unmatchedMen
                unmatchedMen.append(manIndex)
                continue
            #If the women prefers the man :>
            else:
                #Match the man to his next favorite woman
                matching[('m', manIndex)] = ('w', proposeNext)
                matching[('w', proposeNext)] = ('m', manIndex)
                #The rival will, unfortunately, be single
                unmatchedMen.append(rivalIndex)
                continue

In [57]:
#Examine the matching
for match in matching:
    if match[0] == 'm':
        print(f'Man {match[1]} is matched to women {matching[match][1]}')
        print('Woman {} is matched to man {}'.format(matching[match][1], matching[('w',matching[match][1])][1]))

Man 67 is matched to women 67
Woman 67 is matched to man 67
Man 66 is matched to women 10
Woman 10 is matched to man 66
Man 65 is matched to women 60
Woman 60 is matched to man 65
Man 64 is matched to women 44
Woman 44 is matched to man 64
Man 63 is matched to women 1
Woman 1 is matched to man 63
Man 62 is matched to women 66
Woman 66 is matched to man 62
Man 61 is matched to women 17
Woman 17 is matched to man 61
Man 60 is matched to women 39
Woman 39 is matched to man 60
Man 59 is matched to women 33
Woman 33 is matched to man 59
Man 58 is matched to women 50
Woman 50 is matched to man 58
Man 57 is matched to women 12
Woman 12 is matched to man 57
Man 56 is matched to women 58
Woman 58 is matched to man 56
Man 55 is matched to women 5
Woman 5 is matched to man 55
Man 54 is matched to women 29
Woman 29 is matched to man 54
Man 53 is matched to women 14
Woman 14 is matched to man 53
Man 52 is matched to women 2
Woman 2 is matched to man 52
Man 51 is matched to women 30
Woman 30 is matc